In [2]:
import complexnn
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Layer
import keras
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)
np.random.default_rng(7414)

Using TensorFlow backend.


Generator(PCG64) at 0x16875899138

# Inception Layer

In [3]:
class InceptLayer(Layer):
    def __init__(self,
                 filter1,
                 filter2,
                 filter3,
                 filter4,
                 kernel1,
                 kernel2,
                 kernel3,
                 kernel4,
                 num_units=4,
                 name='InceptionLayer',**kwargs):
        super().__init__(**kwargs)
        self.filter1 = filter1
        self.filter2 = filter2
        self.filter3 = filter3
        self.filter4 = filter4
        self.kernel1 = kernel1
        self.kernel2 = kernel2
        self.kernel3 = kernel3
        self.kernel4 = kernel4
        self.num_units = num_units
    def build(self, input_shape):
        self.conv1 = complexnn.conv.ComplexConv2D(self.filter1,self.kernel1,padding='same')
        self.conv2 = complexnn.conv.ComplexConv2D(self.filter2,self.kernel2,padding='same')
        self.conv3 = complexnn.conv.ComplexConv2D(self.filter3,self.kernel3,padding='same')
        self.conv4 = complexnn.conv.ComplexConv2D(self.filter4,self.kernel4,padding='same')
        self.act = AmplitudeMaxout(self.num_units)
    def call(self, x):
        conv_output1 = self.conv1(x)
        real1, imag1 = self.seperate(self.act(conv_output1))
        conv_output2 = self.conv2(x)
        real2, imag2 = self.seperate(self.act(conv_output2))
        conv_output3 = self.conv3(x)
        real3, imag3 = self.seperate(self.act(conv_output3))
        conv_output4 = self.conv4(x)
        real4, imag4 = self.seperate(self.act(conv_output4))
        return tf.concat([real1,real2,real3,real4,imag1,imag2,imag3,imag4], axis=-1)
    def compute_output_shape(self, input_shape):
        last_dim1 = self.compute_shape(self.filter1,self.num_units,self.filter1%self.num_units)
        last_dim2 = self.compute_shape(self.filter2,self.num_units,self.filter2%self.num_units)
        last_dim3 = self.compute_shape(self.filter3,self.num_units,self.filter3%self.num_units)
        last_dim4 = self.compute_shape(self.filter4,self.num_units,self.filter4%self.num_units)
        shape = list(input_shape)
        shape[-1] = last_dim1 + last_dim2 + last_dim3 + last_dim4
        return tuple(shape)
    
    def get_config(self):
        pass
    
    def seperate(self, x):
        shape = x.get_shape().as_list()
        dim = shape[-1]//2
        realpart = x[:,:,:,:dim]
        imagpart = x[:,:,:,dim:]
        return realpart, imagpart
    def compute_shape(self,filters,units,flag):
        if flag:
            return 2*(filters//units + 1)
        else:
            return 2*(filters//units)

# Activation function:AMU

In [4]:
class AmplitudeMaxout(Layer):
    def __init__(self, num_pieces, name='AMU',**kwargs):
        super().__init__(**kwargs)
        self.__name__ = name
        self.num_pieces = num_pieces

    @tf.function
    def call(self, x, axis=None):
        shape = x.get_shape().as_list()
        if axis is None:
            axis = -1
            shape[0] = -1
        if shape[axis]%2:
            raise ValueError(f'nb of real/imaginary channel are inequivalent')
        num_channels = shape[-1]//2
        self.num_units = num_channels//self.num_pieces
        if num_channels%self.num_pieces:
            self.num_units += 1
            num_padding = self.num_pieces - num_channels%self.num_pieces
            padding_size = tf.concat([tf.shape(x)[:-1],tf.constant([num_padding])],axis=-1)
            zero_padding = tf.zeros(padding_size)           
        shape[axis] = self.num_units
        exp_shape = shape + [self.num_pieces]
        real_part = x[:,:,:,:num_channels]
        imag_part = x[:,:,:,num_channels:]        
        if num_channels%self.num_pieces:
            real_part = tf.concat([real_part,zero_padding], axis=-1)
            imag_part = tf.concat([imag_part,zero_padding], axis=-1)
        real_part = tf.reshape(real_part, exp_shape)
        imag_part = tf.reshape(real_part, exp_shape)
        real_part, imag_part = self.return_AMU(real_part, imag_part, exp_shape)
        return tf.concat([real_part,imag_part],axis=-1)         

    def compute_output_shape(self, input_shape):
        shape = list(input_shape)
        shape[-1] = 2*self.num_units
        return tuple(shape)
    
    def get_config(self):
        base_config = super().get_config()
        config = {'units': self.num_units,
                  'pieces': self.num_pieces}
        return dict(list(base_config.items()) + list(config.items()))
    
    def return_AMU(self, real_part, imag_part ,expand_shape):
        modulus = real_part**2 + imag_part**2
        expand_modulus = tf.reshape(modulus, expand_shape)    
        cond = tf.equal(expand_modulus,tf.reduce_max(expand_modulus,axis=-1,keepdims=True))
        real_part = tf.reduce_max(real_part*tf.cast(cond,dtype=tf.float32),axis=-1)
        imag_part = tf.reduce_max(imag_part*tf.cast(cond,dtype=tf.float32),axis=-1)
        return real_part, imag_part

# Loss function: MSE

In [5]:
@tf.function
def ComplexRMS(y_true, y_pred):
    shape = y_pred.get_shape().as_list()
    if shape[-1]%2:
        raise ValueError(f"nb of imaginary part isn't equal to that of real part")
    num_channels = shape[-1]//2
    n_points = 1
    for n in shape[1:]:
        n_points = n_points*n
    n_points = tf.cast(n_points,dtype=tf.float32)/2
    real_pdt = y_pred[:,:,:,num_channels:]
    imag_pdt = y_pred[:,:,:,:num_channels]
    real_true = y_true[:,:,:,num_channels:]
    imag_true = y_true[:,:,:,:num_channels]
    return tf.sqrt(tf.reduce_sum((real_pdt-real_true)**2+(imag_pdt-imag_true)**2)/n_points)
@tf.function
def ComplexMSE(y_true, y_pred):
    shape = y_pred.get_shape().as_list()
    if shape[-1]%2:
        raise ValueError(f"nb of imaginary part isn't equal to that of real part")
    num_channels = shape[-1]//2
    n_points = 1
    for n in shape[1:]:
        n_points = n_points*n
    n_points = tf.cast(n_points,dtype=tf.float32)/2
    real_pdt = y_pred[:,:,:,num_channels:]
    imag_pdt = y_pred[:,:,:,:num_channels]
    real_true = y_true[:,:,:,num_channels:]
    imag_true = y_true[:,:,:,:num_channels]
    return tf.reduce_sum(((real_pdt-real_true)**2+(imag_pdt-imag_true)**2)) / n_points
@tf.function
def ComplexMAE(y_true, y_pred):
    shape = y_pred.get_shape().as_list()
    if shape[-1]%2:
        raise ValueError(f"nb of imaginary part isn't equal to that of real part")
    num_channels = shape[-1]//2
    n_points = 1
    for n in shape[1:]:
        n_points = n_points*n
    n_points = tf.cast(n_points,dtype=tf.float32)/2
    real_pdt = y_pred[:,:,:,num_channels:]
    imag_pdt = y_pred[:,:,:,:num_channels]
    real_true = y_true[:,:,:,num_channels:]
    imag_true = y_true[:,:,:,:num_channels]
    return tf.reduce_sum(tf.sqrt((real_pdt-real_true)**2+(imag_pdt-imag_true)**2)) / n_points


In [6]:
x_train = np.random.rand(61,64,64,32)
y_train = np.random.rand(61,64,64,2)
input_size = x_train.shape[1:]

使用上keras會檢查層與層之間的input_shape與output_shape，若我們將activation放在ComplexConv2D中輸出的channel數會減少，e.g. 設定16 filters輸出32 channel但經過AMU(4)後會剩下8 channels，若下層再增加Conv layer則會跳出輸出channel數8不是32倍數的錯誤資訊，因此受限於AMU只能設定輸出和Conv同輸出# channel，要解決這個問題需要將activation獨立出成一個layer

In [15]:
tf.keras.backend.clear_session()
InputTensor = keras.Input(shape=input_size)
conv1 = complexnn.conv.ComplexConv2D(64,(3,3),padding='same')(InputTensor)
amp1 = AmplitudeMaxout(4)(conv1)
conv2 = complexnn.conv.ComplexConv2D(16,(3,3),padding='same')(amp1)
amp2 = AmplitudeMaxout(4)(conv2)
incep = InceptLayer(filter1=4,
                    filter2=4,
                    filter3=4,
                    filter4=4,
                    kernel1=(3,5),
                    kernel2=(3,3),
                    kernel3=(5,3),
                    kernel4=(3,5),
                    num_units=4)(amp2)
Output = complexnn.conv.ComplexConv2D(1,(1,1),padding='same')(incep)

model = keras.Model(inputs=InputTensor,outputs=Output)
model.built
model.summary()
model.compile(optimizer=keras.optimizers.RMSprop(),loss=ComplexRMS)


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 32)        0         
_________________________________________________________________
complex_conv2d_1 (ComplexCon (None, 64, 64, 128)       18560     
_________________________________________________________________
amplitude_maxout_1 (Amplitud (None, 64, 64, 32)        0         
_________________________________________________________________
complex_conv2d_2 (ComplexCon (None, 64, 64, 32)        4640      
_________________________________________________________________
amplitude_maxout_2 (Amplitud (None, 64, 64, 8)         0         
_________________________________________________________________
incept_layer_1 (InceptLayer) (None, 64, 64, 8)         1760      
_________________________________________________________________
complex_conv2d_7 (ComplexCon (None, 64, 64, 2)         10  

In [16]:
model.fit(x_train,y_train,epochs=20)

Epoch 1/20
61/61 [==============================] - 1s 14ms/step - loss: 4.4889
Epoch 2/20
61/61 [==============================] - 0s 5ms/step - loss: 4.1556
Epoch 3/20
61/61 [==============================] - 0s 5ms/step - loss: 3.5623
Epoch 4/20
61/61 [==============================] - 0s 5ms/step - loss: 2.9734
Epoch 5/20
61/61 [==============================] - 0s 5ms/step - loss: 2.6531
Epoch 6/20
61/61 [==============================] - 0s 5ms/step - loss: 2.4792
Epoch 7/20
61/61 [==============================] - 0s 4ms/step - loss: 2.3845
Epoch 8/20
61/61 [==============================] - 0s 4ms/step - loss: 2.3338
Epoch 9/20
61/61 [==============================] - 0s 5ms/step - loss: 2.3190
Epoch 10/20
61/61 [==============================] - 0s 5ms/step - loss: 2.3612
Epoch 11/20
61/61 [==============================] - 0s 5ms/step - loss: 2.3677
Epoch 12/20
61/61 [==============================] - 0s 4ms/step - loss: 2.3322
Epoch 13/20
61/61 [=============================

In [ ]:
def test_conv_tf(x,w,b):
    x_shape = list(x.shape)
    x_dim = x_shape[-1]//2
    xre = x[:,:,:,:x_dim]
    xim = x[:,:,:,x_dim:]
    
    w_shape = list(w.shape)
    w_dim = w_shape[-1]//2
    new_shape = [w_shape[0],w_shape[1],x_dim,w_dim]
    wre = w[:,:,:,:w_dim].reshape(new_shape)
    wim = w[:,:,:,w_dim:].reshape(new_shape)
    crr = tf.nn.conv2d(xre, wre, strides=1, padding='SAME')
    cii = tf.nn.conv2d(xim, wim, strides=1, padding='SAME')
    cri = tf.nn.conv2d(xre, wim, strides=1, padding='SAME')
    cir = tf.nn.conv2d(xim, wre, strides=1, padding='SAME')
    return tf.nn.bias_add(np.concatenate([crr-cii,cri+cir],axis=-1),b)


In [31]:
def test_conv_keras(x,w,b):
    x_shape = list(x.shape)
    x_dim = x_shape[-1]//2
    xre = x[:,:,:,:x_dim]
    xim = x[:,:,:,x_dim:]
    
    w_shape = list(w.shape)
    w_dim = w_shape[-1]//2
    new_shape = [w_shape[0],w_shape[1],x_dim,w_dim]
    wre = tf.reshape(w[:,:,:,:w_dim],new_shape)
    wim = tf.reshape(w[:,:,:,w_dim:],new_shape)
    
    xre = tf.convert_to_tensor(xre,dtype=tf.float32)
    xim = tf.convert_to_tensor(xim,dtype=tf.float32)
    crr = tf.keras.backend.conv2d(xre, wre, strides=1, padding='same')

    cii = tf.keras.backend.conv2d(xim, wim, strides=1, padding='same')

    cri = tf.keras.backend.conv2d(xre, wim, strides=1, padding='same')

    cir = tf.keras.backend.conv2d(xim, wre, strides=1, padding='same')
    return tf.nn.bias_add(np.concatenate([crr-cii,cri+cir],axis=-1),b)

In [32]:
w_conv1,b_conv1,w_conv2,b_conv2,w_incept1,b_incept1,w_incept2,b_incept2,w_incept3,b_incept3,w_incept4,b_incept4,w_conv3,b_conv3 = model.weights
print('conv1 result: /n')
print(test_conv_keras(x_train,w_conv1,b_conv1))

conv1 result: /n
tf.Tensor(
[[[[-3.93537357e-02  6.24676168e-01  8.25895846e-01 ... -2.81607509e-02
    -5.84610403e-02  2.77851559e-02]
   [-1.93532091e-02  9.89194870e-01  1.25693893e+00 ... -5.68240285e-02
    -1.19689941e-01 -3.37359980e-02]
   [-1.24129103e-02  1.05734217e+00  1.25280106e+00 ... -9.70785022e-02
    -1.29306734e-01  1.05197430e-02]
   ...
   [ 8.43029935e-03  9.83600140e-01  1.24651265e+00 ...  8.97188783e-02
     7.05391169e-03  3.86038125e-02]
   [-8.09813733e-04  1.00609982e+00  1.16426539e+00 ...  5.28244376e-02
    -4.29542065e-02 -3.32555547e-02]
   [-6.04769513e-02  7.49932647e-01  8.17239821e-01 ...  1.58147216e-02
    -4.10859585e-02 -3.71413901e-02]]

  [[-3.89872454e-02  9.72106338e-01  1.13622689e+00 ... -4.53172326e-02
    -9.70706940e-02  2.42076069e-02]
   [-1.60737913e-02  1.54382837e+00  1.77661371e+00 ... -7.46586323e-02
    -1.93008542e-01 -6.69402331e-02]
   [-4.62221168e-02  1.58251536e+00  1.82119942e+00 ... -2.62628794e-01
    -3.41235638e-01

In [25]:
model.weights
print(model.weights)

[<tf.Variable 'complex_conv2d_1/kernel:0' shape=(3, 3, 16, 128) dtype=float32, numpy=
array([[[[-0.00033142,  0.01203605,  0.0095028 , ..., -0.01021151,
          -0.01474319, -0.00200606],
         [ 0.01008349,  0.01003461,  0.00162667, ..., -0.02676732,
          -0.01114028, -0.01922543],
         [-0.00480597,  0.0137211 ,  0.00848179, ..., -0.00803025,
          -0.0184405 , -0.0085149 ],
         ...,
         [-0.00498307, -0.0002731 ,  0.00780383, ..., -0.02721333,
          -0.02025492,  0.00503019],
         [ 0.00736294,  0.01928953,  0.00013483, ..., -0.01074655,
          -0.01865556,  0.00522701],
         [-0.01296402,  0.00617582,  0.01128132, ..., -0.0153293 ,
          -0.01218642,  0.00316262]],

        [[ 0.00057578,  0.01374798, -0.00243189, ..., -0.01623419,
          -0.01786762, -0.00457995],
         [ 0.0010782 ,  0.022293  ,  0.01372092, ..., -0.02606141,
          -0.02426114,  0.00387559],
         [ 0.00923553,  0.00265249,  0.00653906, ..., -0.02488136,

In [28]:
*gg = (1,2,3)

SyntaxError: starred assignment target must be in a list or tuple (<ipython-input-28-985668686070>, line 4)